## Imports

In [1]:
from functools import cache

from pyspark.sql import SparkSession

spark = (SparkSession
            .builder
            .appName('Beyon-Challenge')
            .config('spark.sql.ansi.enabled', 'false')
            .getOrCreate())

spark.sparkContext.setLogLevel('ERROR')

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/17 15:46:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DateType, IntegerType, BooleanType, LongType
from pyspark.sql.functions import col, udf
from pyspark import pandas as pd

from brazilcep import get_address_from_cep

from functools import cache

/home/arthur/workspace/beyon_challenge/.venv/lib/python3.13/site-packages/pyspark/pandas/__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


## Carregar e Explorar

In [3]:
metas = spark.read.csv('metas por marca.csv', sep=';', header=True)
metas.show(10)

+----------+-----+----+
|     Marca| Meta| Ano|
+----------+-----+----+
|  Heineken|25000|2024|
|   Devassa| 7000|2024|
|    Amstel| 3000|2024|
|     Schin| 3000|2024|
|   Glacial|  300|2024|
|Viva Schin|  500|2024|
| Eisenbahn| 3000|2024|
|  Itubaina|  500|2024|
|Agua Schin|  500|2024|
|  Bonafont|  500|2024|
+----------+-----+----+
only showing top 10 rows


In [4]:
metas.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Marca   20 non-null     object
 1   Meta    20 non-null     object
 2   Ano     20 non-null     object
dtypes: object(3)
memory usage: 612.0+ bytes


In [5]:
vendas = pd.read_excel('sales.xlsx', header=0)
vendas.info()

/home/arthur/workspace/beyon_challenge/.venv/lib/python3.13/site-packages/pyspark/pandas/namespace.py:1133: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  return pd.read_excel(


<class 'pyspark.pandas.frame.DataFrame'>
Index: 28884 entries, 0 to 28883
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Código do Cliente  28884 non-null  int64  
 1   cod material       28884 non-null  int64  
 2   b2b_status         28853 non-null  object 
 3   volume_hl          28884 non-null  float64
 4   valor              28884 non-null  float64
 5   data_doc           28884 non-null  object 
 6   materialPackaging  28861 non-null  float64
 7   brand              28861 non-null  float64
 8   brand_desc         28606 non-null  object 
 9   material           28606 non-null  object 
 10  CEP_cliente        28884 non-null  int64  
dtypes: float64(4), int64(3), object(4)

In [6]:
vendas = vendas.to_spark()
vendas.schema

/home/arthur/workspace/beyon_challenge/.venv/lib/python3.13/site-packages/pyspark/pandas/utils.py:1037: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


StructType([StructField('Código do Cliente', LongType(), False), StructField('cod material', LongType(), False), StructField('b2b_status', BooleanType(), True), StructField('volume_hl', DoubleType(), False), StructField('valor', DoubleType(), False), StructField('data_doc', StringType(), False), StructField('materialPackaging', DoubleType(), True), StructField('brand', DoubleType(), True), StructField('brand_desc', StringType(), True), StructField('material', StringType(), True), StructField('CEP_cliente', LongType(), False)])

In [7]:
# vendas.show(10)

### Correção de Schema de Vendas

In [8]:

venda_schema = StructType([
    StructField('Código do Cliente', LongType(), True),
    StructField('cod material', LongType(), True),
    StructField('b2b_status', BooleanType(), True),
    StructField('volume_hl', FloatType(), True),
    StructField('valor', FloatType(), True),
    StructField('data_doc', DateType(), True),
    StructField('materialPackaging', IntegerType(), True),
    StructField('brand', IntegerType(), True),
    StructField('brand_desc', StringType(), True),
    StructField('material', StringType(), True),
    StructField('CEP_cliente', IntegerType(), True),
])

vendas = vendas.select(
    [col(field.name).cast(field.dataType) for field in venda_schema.fields]
)

vendas.schema


StructType([StructField('Código do Cliente', LongType(), False), StructField('cod material', LongType(), False), StructField('b2b_status', BooleanType(), True), StructField('volume_hl', FloatType(), False), StructField('valor', FloatType(), False), StructField('data_doc', DateType(), True), StructField('materialPackaging', IntegerType(), True), StructField('brand', IntegerType(), True), StructField('brand_desc', StringType(), True), StructField('material', StringType(), True), StructField('CEP_cliente', IntegerType(), False)])

In [9]:
vendas.show(10)

+-----------------+------------+----------+---------+---------+----------+-----------------+-----+-------------+--------+-----------+
|Código do Cliente|cod material|b2b_status|volume_hl|    valor|  data_doc|materialPackaging|brand|   brand_desc|material|CEP_cliente|
+-----------------+------------+----------+---------+---------+----------+-----------------+-----+-------------+--------+-----------+
|         26349684|     2708451|      true|   1.4616| 33.32308|2020-01-08|                2|  116|   Viva Schin|     Pet|   70344080|
|         19620228|     2709138|      true| 0.630432|143.38814|2020-01-08|                4|   42|Schin No Grau|   Vidro|   65065210|
|         13021080|      657999|      true| 0.066933|  99.9922|2020-01-08|                4|    8|       Dreher|   Vidro|   41340170|
|         12981552|     2707296|      true|    1.386|1106.6144|2020-01-08|                4|   33|        Schin|   Vidro|   89805517|
|         12402084|     2701086|      true| 0.088722| 22.61722

Exception ignored in: <_io.BufferedWriter name=6>
Traceback (most recent call last):
  File "/home/arthur/workspace/beyon_challenge/.venv/lib/python3.13/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 200, in manager
BrokenPipeError: [Errno 32] Broken pipe


### Exploração:

In [10]:
total_clientes = vendas.select('Código do Cliente').distinct().count()
print(f'Total de clientes: {total_clientes}')

Total de clientes: 58


In [11]:
b2b_nulos = vendas.select('*').where(col('b2b_status').isNull()).count()
print(f'Casos b2b_status desconhecido: {b2b_nulos}')

mp_nulo = vendas.select('*').where(col('materialPackaging').isNull()).count()
print(f'Casos materialPackaging desconhecido: {mp_nulo}')

brand_nulo = vendas.select('*').where(col('brand').isNull()).count()
print(f'Casos brand desconhecido: {brand_nulo}')

bd_nulo = vendas.select('*').where(col('brand_desc').isNull()).count()
print(f'Casos brand_desc desconhecido: {bd_nulo}')

material_nulo = vendas.select('*').where(col('material').isNull()).count()
print(f'Casos material desconhecido: {material_nulo}')

data_doc_nulo = vendas.select('*').where(col('data_doc').isNull()).count()
print(f'Casos date desconhecido: {data_doc_nulo}')

Casos b2b_status desconhecido: 31
Casos materialPackaging desconhecido: 23
Casos brand desconhecido: 23
Casos brand_desc desconhecido: 278
Casos material desconhecido: 278
Casos date desconhecido: 2689


Valor e Volume parecem estar zerados em casos em que outras colunas estão nulas

In [12]:
vendas.select('*').where(col('materialPackaging').isNull()).show()

+-----------------+------------+----------+---------+-----+----------+-----------------+-----+----------+--------+-----------+
|Código do Cliente|cod material|b2b_status|volume_hl|valor|  data_doc|materialPackaging|brand|brand_desc|material|CEP_cliente|
+-----------------+------------+----------+---------+-----+----------+-----------------+-----+----------+--------+-----------+
|         19620228|     1847970|      true|      0.0|  0.0|2023-12-18|             NULL| NULL|      NULL|    NULL|   65065210|
|         16380564|     1847943|      true|      0.0|  0.0|2023-12-11|             NULL| NULL|      NULL|    NULL|   41340170|
|         16380564|     1847955|      true|      0.0|  0.0|2023-12-11|             NULL| NULL|      NULL|    NULL|   41340170|
|         25631928|     1847949|      true|      0.0|  0.0|2023-12-11|             NULL| NULL|      NULL|    NULL|   65065210|
|         16380564|     1847949|      true|      0.0|  0.0|2023-12-11|             NULL| NULL|      NULL|    NU

In [13]:
vendas.select('*').where("valor=0 and volume_hl=0").show()

+-----------------+------------+----------+---------+-----+----------+-----------------+-----+----------+--------+-----------+
|Código do Cliente|cod material|b2b_status|volume_hl|valor|  data_doc|materialPackaging|brand|brand_desc|material|CEP_cliente|
+-----------------+------------+----------+---------+-----+----------+-----------------+-----+----------+--------+-----------+
|         12981552|     1058727|      true|      0.0|  0.0|2020-11-19|                0|    0|      NULL|    NULL|   89805517|
|          2472120|     2711715|      true|      0.0|  0.0|2020-10-27|                8|   65| Lagunitas|  Barril|   65065210|
|         27607140|     1058226|      true|      0.0|  0.0|2020-11-25|                0|    0|      NULL|    NULL|   54530108|
|         26281800|     2709378|      true|      0.0|  0.0|2020-10-22|                7|  107|   Devassa|    Lata|   60752360|
|         12981552|     2707296|      true|      0.0|  0.0|2020-10-20|                4|   33|     Schin|   Vid

Saber o significado de "volume_hl" ajudaria na análise.

## Floco de Neve

Tabela Fato:
   - Vendas

Tabelas Dimensão:
   - Data ✓
   - Cliente ✓
   - Material ✓

Sub-Dimensão:
   - Geografia ✓
   - Marca (Brand) ✓

In [14]:
vendas.createOrReplaceTempView('vendas')

In [15]:
dim_date = spark.sql(
    '''
    SELECT
        DISTINCT data_doc AS date_complete,
        WEEKDAY(data_doc) AS _weekday,
        DAY(data_doc) AS _day,
        MONTH(data_doc) AS _month,
        YEAR(data_doc) AS _year,
        CASE
            WHEN MONTH(data_doc) BETWEEN 1 AND 3
            THEN 1
            WHEN MONTH(data_doc) BETWEEN 4 AND 6
            THEN 2
            WHEN MONTH(data_doc) BETWEEN 7 AND 9
            THEN 3
            ELSE 4
        END AS trimester,
        CAST(DATE_FORMAT(data_doc, "yyyyMMdd") AS INTEGER) AS pk_date
    FROM vendas
    WHERE data_doc IS NOT NULL
    --ORDER BY trimester ASC
    '''
)
dim_date.show(3)

+-------------+--------+----+------+-----+---------+--------+
|date_complete|_weekday|_day|_month|_year|trimester| pk_date|
+-------------+--------+----+------+-----+---------+--------+
|   2020-01-16|       3|  16|     1| 2020|        1|20200116|
|   2020-10-14|       2|  14|    10| 2020|        4|20201014|
|   2020-09-02|       2|   2|     9| 2020|        3|20200902|
+-------------+--------+----+------+-----+---------+--------+
only showing top 3 rows


In [16]:
dim_geo = spark.sql(
    '''
    SELECT
        DISTINCT CEP_cliente as cep-- SK
    FROM
        vendas
    WHERE
        CEP_cliente IS NOT NULL
    --ORDER BY brand ASC
    '''
)

dim_geo.show(3)

+--------+
|     cep|
+--------+
|69099244|
|75101491|
|70344080|
+--------+
only showing top 3 rows


In [17]:
address_schema = StructType([
    StructField('cidade', StringType(), True),
    StructField('estado', StringType(), True)
])

def get_address(cep):
    try:
        endereco = get_address_from_cep(f'{cep:08d}')
        return {'cidade': endereco['city'], 'estado': endereco['uf']}
    except:
        pass
    return {'cidade': None, 'estado': None}

get_address_udf = udf(get_address, address_schema)

Esta **NÃO** é a melhor forma! Aqui estamos fazendo 2 chamadas por linha. Além do risco de sobrecarregar a API... mas para demonstração, é suficiente.

In [18]:
dim_geo = (dim_geo
           .withColumn('cidade', get_address_udf(col('cep'))['cidade'])
           .withColumn('estado', get_address_udf(col('cep'))['estado'])
           )
dim_geo.show(3)

+--------+--------+------+
|     cep|  cidade|estado|
+--------+--------+------+
|69099244|  Manaus|    AM|
|75101491|Anápolis|    GO|
|70344080|Brasília|    DF|
+--------+--------+------+
only showing top 3 rows


In [19]:
spark.sql('''
    SELECT
        DISTINCT `Código do Cliente` AS cod_client,
        collect_set(CEP_cliente) as lista_ceps
    FROM vendas
    GROUP BY
        cod_client
    HAVING size(collect_set(CEP_cliente)) > 1
''').show(100)

+----------+----------+
|cod_client|lista_ceps|
+----------+----------+
+----------+----------+



Todos os clientes tem apenas 1 CEP, mas poderíamos ter uma lista por cliente se houvessem mais

In [20]:
dim_client = spark.sql(
    '''
    SELECT
        DISTINCT `Código do Cliente` as cod_client, -- PK
        CEP_cliente AS sk_cep
    FROM
        vendas
    WHERE
        `Código do Cliente` IS NOT NULL
    --ORDER BY brand ASC
    '''
)

dim_client.show(3)

+----------+--------+
|cod_client|  sk_cep|
+----------+--------+
|  27828336|14021020|
|  19108092|65065210|
|  14345688|66030760|
+----------+--------+
only showing top 3 rows


In [21]:
dim_material = spark.sql(
    '''
    SELECT
        DISTINCT `cod material` AS cod_material, --PK
        material AS material_desc
    FROM vendas
    ORDER BY material_desc ASC
    '''
)
dim_material.show(3)

+------------+-------------+
|cod_material|material_desc|
+------------+-------------+
|     1840752|         NULL|
|     1847970|         NULL|
|     1840734|         NULL|
+------------+-------------+
only showing top 3 rows


In [22]:
dim_brand = spark.sql(
    '''
    SELECT
        DISTINCT brand, -- SK
        brand_desc
    FROM
        vendas
    WHERE
        brand IS NOT NULL
        AND brand_desc IS NOT NULL
    --ORDER BY brand ASC
    '''
)

dim_brand.show(3)

+-----+----------+
|brand|brand_desc|
+-----+----------+
|   17|    Amstel|
|  105|  Bonafont|
|   52| Eisenbahn|
+-----+----------+
only showing top 3 rows


In [23]:
fato =  (vendas.selectExpr(
                        'volume_hl',
                        'valor',
                        'b2b_status',
                        '`Código do Cliente` as fk_cliente',
                        'CAST(DATE_FORMAT(data_doc, "yyyyMMdd") AS INTEGER) AS fk_date',
                        'brand as fk_brand')
         .where(col('valor')>0)
         .where(col('volume_hl')>0)
         .where(col('b2b_status').isNotNull())
         .where(col('fk_cliente').isNotNull())
         .where(col('fk_date').isNotNull())
         .where(col('fk_brand').isNotNull())
         )


fato.show(3)
fato.count()

+---------+---------+----------+----------+--------+--------+
|volume_hl|    valor|b2b_status|fk_cliente| fk_date|fk_brand|
+---------+---------+----------+----------+--------+--------+
|   1.4616| 33.32308|      true|  26349684|20200108|     116|
| 0.630432|143.38814|      true|  19620228|20200108|      42|
| 0.066933|  99.9922|      true|  13021080|20200108|       8|
+---------+---------+----------+----------+--------+--------+
only showing top 3 rows


Exception ignored in: <_io.BufferedWriter name=6>
Traceback (most recent call last):
  File "/home/arthur/workspace/beyon_challenge/.venv/lib/python3.13/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 200, in manager
BrokenPipeError: [Errno 32] Broken pipe


22733

In [24]:
fato.cache()
fato.createOrReplaceTempView('fato')

dim_geo.createOrReplaceTempView('dim_geo')
dim_date.createOrReplaceTempView('dim_date')
dim_brand.createOrReplaceTempView('dim_brand')
dim_material.createOrReplaceTempView('dim_material')
dim_brand.createOrReplaceTempView('dim_brand')
dim_client.createOrReplaceTempView('dim_client')

Médias (3 meses)

In [25]:
spark.sql(
    '''
    WITH VendasComDataMaxima AS (
    SELECT
        f.fk_cliente,
        f.valor,
        f.volume_hl,
        f.b2b_status,
        d.date_complete,
        MAX(d.date_complete) OVER (PARTITION BY f.fk_cliente) AS max_venda_date
    FROM
        fato f
    JOIN
        dim_date d ON f.fk_date = d.pk_date
    )

    SELECT
        fk_cliente,
        ROUND(MEAN(valor), 2) AS valor_medio,
        ROUND(MEAN(volume_hl), 2) AS volume_medio
    FROM
        VendasComDataMaxima
    WHERE
        date_complete >= ADD_MONTHS(max_venda_date, -3)
        --AND b2b_status = True
    GROUP BY
        fk_cliente
    ORDER BY volume_medio DESC
    '''
).show(58)

+----------+-----------+------------+
|fk_cliente|valor_medio|volume_medio|
+----------+-----------+------------+
|  28663752|   59131.02|       58.52|
|  19241664|   19540.83|       23.27|
|  21093792|   22011.83|        22.0|
|  21076056|    6172.55|       12.06|
|   2449428|    15733.7|       11.79|
|  25538628|    7620.89|        6.78|
|  24200784|    8159.72|        5.92|
|  27224268|    3797.18|        2.92|
|  24691824|    1683.55|         2.9|
|  30733116|    3857.31|         2.8|
|  31209876|    1958.47|         2.1|
|  25852692|    3241.81|        2.09|
|  19560132|    3069.97|        2.02|
|  26349684|    1862.86|        1.89|
|  21093336|    1331.65|        1.81|
|  20350824|    1671.08|        1.73|
|  26281800|    1405.75|        1.73|
|  12981552|    1824.49|        1.66|
|  26543724|    2401.19|         1.6|
|  19108092|    1407.36|        1.57|
|  20869260|    2458.58|        1.44|
|  20704572|    1021.26|        1.39|
|  28518780|    2481.23|        1.37|
|  27607140|

Médias (6 meses)

In [26]:
spark.sql(
    '''
    WITH VendasComDataMaxima AS (
    SELECT
        f.fk_cliente,
        f.valor,
        f.volume_hl,
        f.b2b_status,
        d.date_complete,
        MAX(d.date_complete) OVER (PARTITION BY f.fk_cliente) AS max_venda_date
    FROM
        fato f
    JOIN
        dim_date d ON f.fk_date = d.pk_date
    )

    SELECT
        fk_cliente,
        ROUND(MEAN(valor), 2) AS valor_medio,
        ROUND(MEAN(volume_hl), 2) AS volume_medio
    FROM
        VendasComDataMaxima
    WHERE
        date_complete >= ADD_MONTHS(max_venda_date, -6)
        -- AND b2b_status = True
    GROUP BY
        fk_cliente
    ORDER BY volume_medio DESC
    '''
).show(58)

+----------+-----------+------------+
|fk_cliente|valor_medio|volume_medio|
+----------+-----------+------------+
|  28663752|   55994.68|       54.32|
|  21093792|   21670.92|       22.33|
|  19241664|   11836.56|       16.38|
|   2449428|   20314.32|        15.7|
|  21076056|    7124.52|       10.11|
|  25538628|   13048.87|        7.32|
|  30733116|    6032.51|        4.42|
|  24200784|    5751.88|        4.41|
|  21093336|    2178.88|        3.03|
|  27224268|    4853.65|        2.64|
|  21251112|    1987.19|        2.46|
|  24691824|    1624.87|        2.36|
|  31209876|    1864.09|        2.14|
|  18048060|    2125.62|         2.0|
|  12981552|    1964.13|        1.77|
|  19560132|     2517.7|        1.77|
|  26281800|    1442.12|        1.75|
|  25852692|    2462.16|        1.63|
|  26349684|    1479.81|         1.6|
|  20350824|    1448.86|        1.59|
|  19108092|     1508.6|        1.58|
|  27393336|    1844.83|        1.47|
|  26543724|    1916.05|        1.35|
|   2472120|

Share B2B (3 meses)

In [40]:
spark.sql(
    '''
    WITH
        fato_datado AS(
            SELECT
                f.*,
                d._year,
                d._month
            FROM fato f
            JOIN dim_date d
            ON d.pk_date = f.fk_date
        ),

        TotaisB2BPorMes AS (
            SELECT
                fk_cliente,
                _year,
                _month,
                SUM(valor) AS valor_mensal_b2b
            FROM fato_datado
            WHERE b2b_status = TRUE
            GROUP BY fk_cliente, _year, _month
        ),

        TotaisGeraisPorMes AS (
            SELECT
                fk_cliente,
                _year,
                _month,
                SUM(valor) AS valor_mensal_total
            FROM fato_datado
            GROUP BY fk_cliente, _year, _month
        )

        SELECT
            m_b2b.fk_cliente,
            m_b2b._year,
            m_b2b._month,
            CONCAT(CAST(ROUND((m_b2b.valor_mensal_b2b / m_total.valor_mensal_total) * 100,2) AS STRING), '%') AS share_b2b_mensal
        FROM
            TotaisB2BPorMes m_b2b
        JOIN TotaisGeraisPorMes m_total ON
            m_b2b.fk_cliente = m_total.fk_cliente AND
            m_b2b._year = m_total._year AND
            m_b2b._month = m_total._month

    '''
).show(58)

+----------+-----+------+----------------+
|fk_cliente|_year|_month|share_b2b_mensal|
+----------+-----+------+----------------+
|  27489360| 2023|     3|          90.27%|
|  20869260| 2023|     9|          98.29%|
|  18048060| 2022|    10|          100.0%|
|  30795300| 2023|     1|          89.78%|
|  20350824| 2023|    12|          85.27%|
|  21220308| 2023|     7|          30.07%|
|  13426188| 2021|     1|           67.4%|
|  19455504| 2023|     8|          61.76%|
|  26281800| 2021|    10|          93.09%|
|  30795300| 2023|    11|          80.88%|
|  28663752| 2023|     2|          100.0%|
|  29596344| 2024|     1|          88.43%|
|  26543724| 2020|     1|          100.0%|
|  19560132| 2023|    11|          100.0%|
|  26281800| 2023|    10|          97.54%|
|  18048060| 2022|    11|          97.39%|
|  25631928| 2021|    10|          100.0%|
|  18088944| 2020|     4|          77.05%|
|  27489360| 2020|     1|          96.32%|
|  24691824| 2023|     9|          97.51%|
|  27393336